# Single Machine Weighted Tardiness Scheduling Problem

This Jupyter Notebook demonstrates how to solve the **single machine weighted tardiness scheduling problem** using a Mixed-Integer Programming (MIP) model. The optimization is performed using Gurobi.


## Import Libraries

We import the required libraries, including:
- **`gurobipy`** for optimization.
- **`random`** to generate input data.

In [19]:
from gurobipy import Model, GRB
import random

## Define Input Data

We define the following inputs:
1. **Number of jobs (`n`)**: Total jobs to schedule.
2. **Processing times (`p`)**: Randomly generated from a uniform distribution between 1 and 8.
3. **Weights (`w`)**: Randomly generated from a uniform distribution between 1 and 6.
4. **Due dates (`d`)**: Randomly generated from a uniform distribution between 5 and 30.
5. **Release times (`r`)**: All set to 0.

A **Big M** value is calculated as the sum of all processing times.


In [22]:
# Number of jobs
n = 10

# Randomly generate processing times (U(1, 8))
p = [random.randint(1, 8) for _ in range(n)]

# Randomly generate weights (U(1, 6))
w = [random.randint(1, 6) for _ in range(n)]

# Randomly generate due dates (U(5, 30))
d = [random.randint(5, 30) for _ in range(n)]

# Set release times to zero for all jobs
r = [0] * n

# Big M value (can be the sum of all processing times)
M = sum(p)

# Print the input data
print("Processing times (p):", p)
print("Weights (w):", w)
print("Due dates (d):", d)
print("Release times (r):", r)


Processing times (p): [6, 3, 1, 5, 8, 3, 2, 4, 4, 8]
Weights (w): [3, 4, 1, 5, 5, 3, 3, 4, 1, 4]
Due dates (d): [18, 9, 19, 12, 14, 6, 18, 22, 12, 13]
Release times (r): [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## Define and Solve the MIP Model

Using Gurobi, we define the following:
1. **Decision Variables**:
    - Start times (`S`): When each job starts.
    - Completion times (`C`): When each job finishes.
    - Tardiness (`T`): The lateness of each job relative to its due date.
    - Binary variables (`Y_{i,j}`):: 
    - \( Y_{i,j} = 1 \) if job \( i \) is processed before job \( j \), 
    - \( Y_{i,j} = 0 \) otherwise.
2. **Objective**: Minimize the total weighted tardiness.
3. **Constraints**:
    - Completion time = Start time + Processing time.
    - Start time ≥ Release time.
    - Tardiness ≥ Max(0, Completion time - Due date).
    - Non-overlapping jobs using Big-M constraints.


In [37]:
# Create a new model
model = Model("Single Machine Weighted Tardiness Scheduling")

# Decision variables
S = model.addVars(n, vtype=GRB.CONTINUOUS, name="S")  # Start times
C = model.addVars(n, vtype=GRB.CONTINUOUS, name="C")  # Completion times
T = model.addVars(n, vtype=GRB.CONTINUOUS, name="T")  # Tardiness
y = model.addVars(n, n, vtype=GRB.BINARY, name="y")  # Job order binary variables

# Objective function: Minimize total weighted tardiness
model.setObjective(sum(w[i] * T[i] for i in range(n)), GRB.MINIMIZE)

# Constraints
# Completion time constraints
for i in range(n):
    model.addConstr(C[i] == S[i] + p[i], name=f"Completion_{i}")

# Release time constraints
for i in range(n):
    model.addConstr(S[i] >= r[i], name=f"Release_{i}")

# Tardiness constraints
for i in range(n):
    model.addConstr(T[i] >= C[i] - d[i], name=f"Tardiness_{i}")
    model.addConstr(T[i] >= 0, name=f"Non_negative_tardiness_{i}")

# Non-overlapping constraints (Big-M constraints)
for i in range(n):
    for j in range(i + 1, n):
        model.addConstr(S[j] >= C[i] - M * (1 - y[i, j]), name=f"No_overlap_{i}_{j}_1")
        model.addConstr(S[i] >= C[j] - M * y[i, j], name=f"No_overlap_{i}_{j}_2")

# Optimize the model
model.optimize()


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i9-13900H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 130 rows, 130 columns and 330 nonzeros
Model fingerprint: 0x82be42c5
Variable types: 30 continuous, 100 integer (100 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+01]
  Objective range  [1e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 30 rows and 65 columns
Presolve time: 0.00s
Presolved: 100 rows, 65 columns, 290 nonzeros
Variable types: 20 continuous, 45 integer (45 binary)
Found heuristic solution: objective 440.0000000
Found heuristic solution: objective 439.0000000

Root relaxation: objective 0.000000e+00, 42 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth Int

## Extract and Print Results

Once the optimization is complete, we extract:
- Start times (`S`).
- Completion times (`C`).
- Tardiness (`T`).
- Job order (based on start times).


In [40]:
# Extract the job sequence
job_sequence = sorted(range(n), key=lambda i: S[i].x)
job_order = [0] * n
for position, job_index in enumerate(job_sequence):
    job_order[job_index] = position + 1

# Print the results
print("\nResults:")
for i in range(n):
    print(
        f"Job {i + 1}: Start time = {S[i].x:.2f}, Completion time = {C[i].x:.2f}, Tardiness = {T[i].x:.2f}"
    )

print("\nJob Order:", job_order)


Results:
Job 1: Start time = 26.00, Completion time = 32.00, Tardiness = 14.00
Job 2: Start time = 0.00, Completion time = 3.00, Tardiness = 0.00
Job 3: Start time = 25.00, Completion time = 26.00, Tardiness = 7.00
Job 4: Start time = 6.00, Completion time = 11.00, Tardiness = 0.00
Job 5: Start time = 11.00, Completion time = 19.00, Tardiness = 5.00
Job 6: Start time = 3.00, Completion time = 6.00, Tardiness = 0.00
Job 7: Start time = 19.00, Completion time = 21.00, Tardiness = 3.00
Job 8: Start time = 21.00, Completion time = 25.00, Tardiness = 3.00
Job 9: Start time = 40.00, Completion time = 44.00, Tardiness = 32.00
Job 10: Start time = 32.00, Completion time = 40.00, Tardiness = 27.00

Job Order: [8, 1, 7, 3, 4, 2, 5, 6, 10, 9]
